# **Exploring Top Restaurants in Las Vegas: A Data Science Journey!**



## **Background:**
As we gear up for our summer trip to Las Vegas, we've decided to delve into the city's vibrant dining scene. To ensure we experience the best and most budget-friendly eateries, we've utilized the [Yelp dataset](https://business.yelp.com/data/resources/open-dataset/) to identify top-rated restaurants in Las Vegas. We've also filtered out the top 50 fast-food chains to avoid the usual options and embrace more unique culinary experiences.​

### **Project Goal/Stakes:**
Our objective is to analyze the Yelp dataset to pinpoint exceptional yet affordable dining establishments in Las Vegas. By focusing on local favorites and hidden gems, we aim to curate a list of restaurants that offer both quality and value, ensuring our trip is both delicious and economical.​

### **Project Submission:**
A Note from your Backpacking Partner:

"Hello, team. I've gathered and preprocessed the Yelp dataset for Las Vegas, filtering out the top 50 fast-food chains to ensure we explore the city's unique culinary offerings. Now, it's up to you to analyze this data and find the best budget-friendly restaurants for our trip. I request that you do NOT delete/modify any part of the question description, given code, and pre-loaded comments in the code block.

We look forward to your insightful analyses and recommendations for our culinary adventure in Las Vegas! Hope we can have a wonderful trip in Vegas."


Just a reminder, for a better a smooth autograder experience, please:
- Write your answers between <font color='green'>`# ANSWER STARTING HERE`</font> and <font color='green'>`# ANSWER ENDS HERE`</font>.
- Do not delete helper cells
- Always use `seed=42` as your `random_state` in the whole notebook.

Submit this Google Colab/Jupyter Notebook file on Gradescope. Cheers!🥂


### **Part A (4 Pts Total)**

#### You are expected to load the Yelp data and provide some metrics so I know what we are working with. Follow these steps.

##### ***1.1 Import Libraries (0 pt)***

---


Many libraries are imported for you. Import the additional **Sci-kit learn**  modules:
- **Decision Tree Classifier**
- **K Neighbors Classifier**
- **Logistic Regression**
- **Random Forest Classifier**

In [ ]:
# Necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, silhouette_score

import matplotlib.pyplot as plt
import matplotlib
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

from matplotlib.colors import Normalize
from matplotlib.lines import Line2D

import ast
pd.set_option('display.max_columns', None)
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
import seaborn as sns
import warnings # To suppress some warnings

# Suppress the specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

# Add the classifiers to the imports here:
# ANSWER STARTING HERE
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# ANSWER ENDS HERE


##### ***1.2 Understanding the Dataset (1 pt)***

---


1. Load the dataset to a dataframe named `yelp_data`.
2. Print the dimensions of the dataset.
3. Print the column names of the dataframe.

In [ ]:
# Load the yelp dataset
# ANSWER STARTING HERE
yelp_data =

# ANSWER ENDS HERE

In [ ]:
yelp_data.head()

##### ***1.3 Explore the Data (3 pts)***

---


Let's see what is stored in the dataset first to get a feel for what we can expect. Write only a **single line** of code for each.
1. Print the number of **samples** in the dataset (a numerical value).
2. Print the number of **features** in the dataset (a numerical value)
3. **Create two lists of the unique business names and categories**, named `names` and `categories`, respectively.

Your answer should be similar to:
- Business: ["GameWorks", "Divine Cafe", "Trattoria Italia", "Water", ...]
- Rarities: ["Standard", "Mythic", "Legendary", "Ultra-Beast"]


In [ ]:
# Print the number of samples
# ANSWER STARTING HERE


# ANSWER ENDS HERE

# Print the number of features
# ANSWER STARTING HERE


# ANSWER ENDS HERE

# Write code to get your two lists.
# ANSWER STARTING HERE
names =

categories=

# ANSWER ENDS HERE

print("Business:", names)
print(len(names))
print("Categories:", categories)
print(len(categories))


### **Part B (9 Pts Total)**

#### Now we need to do some scraping of the dataset so you can feed it into the training model.

##### ***2.1 Preprocessing (3 pts)***

---
Before training the model, we need to clean and prepare the data. Please do the following.
1. Print the number of missing data in each column. Please specify which column name you got the number from. You can create a dictionary `missing_data`.
2. Handle the missing values in the `yelp_data` dataset the best way you can think of. Save the processed file as `yelp_data`.

*Suggestions:* Fill in empty with 0s, or do some type of imputation technique such as fill in the empty data with the most common or average of data in that column, etc.

In [ ]:
# Print the number of missing data in each column.
# ANSWER STARTING HERE

missing_data =

# ANSWER ENDS HERE


# Write a code that handles missing values in the dataset by discarding them
# ANSWER STARTING HERE

yelp_data =

# ANSWER ENDS HERE


##### ***2.2 Splitting, Training, and Testing (6 pts)***

---

Typically in machine learning, we wish to split the data into various portions in order to let a machine learning model train itself on one part of the data and test itself on another part of the data. Splitting also helps prevent overfitting, ensuring the model can make accurate predictions on new, unseen data.

Training data is used to learn the model, while testing data assesses its generalization to unseen examples. I would recommend using **80%** of the data  for training and **20%** for testing.

Before we split the data, we are going to help you process the raw data a little bit more:
 1. We prefer restaurants close to Canyon Gate in Las Vegas, near our AirBnb.
 2. We add a `preference_rating` column at the end of the dataframe, so that we can predict the rating classification.
 3. We add non-linearity to `preference_rating`

In [ ]:
# Helper Function, Do Not Remove
import numpy as np
import pandas as pd
import hashlib
from sklearn.preprocessing import MinMaxScaler

def business_id_to_float(business_id):
    hash_object = hashlib.md5(business_id.encode())
    hash_hex = hash_object.hexdigest()
    hash_int = int(hash_hex[:8], 16)
    return hash_int / (2**32 - 1)

def float_to_business_id(unique_float, original_business_ids):
    id_to_float = {id: business_id_to_float(id) for id in original_business_ids}
    closest_id = min(id_to_float.keys(), key=lambda x: abs(id_to_float[x] - unique_float))
    return closest_id

# Non-linear transformation functions
def apply_non_linear_transformation(x, steepness=5):
    """
    Apply a sigmoid-like non-linear transformation
    - Increases sensitivity around mid-range values
    - Compresses extreme values
    """
    return 1 / (1 + np.exp(-steepness * (x - 0.5))) * 10

def exponential_boost(x, base=2):
    """
    Apply exponential boosting to favor higher values
    """
    return np.power(x, base)

def angular_score_adjustment(x):
    """
    Use trigonometric function to add non-linear variation
    """
    return np.sin(np.pi * x / 2) * 10

# Define weights
weights = {
    'location_score': 0.25,
    'business_stars': 0.25,
    'reviewer_stars': 0.25,
    'engagement_score': 0.25
}

# Define the target location as Canyon Gate, Las Vegas
target_location = (36.1433, -115.2821)

# Haversine distance calculation
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# Main processing function
def generate_preference_rating(yelp_data):
    # Calculate distance from target location
    yelp_data['distance_km'] = yelp_data.apply(
        lambda row: haversine(target_location[0], target_location[1],
                               row['latitude'], row['longitude']), axis=1
    )

    # Normalize distance to [0, 1] and invert (closer is better)
    scaler = MinMaxScaler()
    yelp_data['location_score'] = 1 - scaler.fit_transform(yelp_data[['distance_km']])

    # Normalize business_stars and reviewer_stars to [0, 1]
    yelp_data['business_stars_norm'] = yelp_data['business_stars'] / 5
    yelp_data['reviewer_stars_norm'] = yelp_data['reviewer_stars'] / 5

    # Calculate engagement_score and normalize
    yelp_data['engagement_score_raw'] = yelp_data['useful'] + yelp_data['funny'] + yelp_data['cool']
    yelp_data['engagement_score'] = scaler.fit_transform(yelp_data[['engagement_score_raw']])

    # Calculate preference_rating with non-linear transformations
    yelp_data['preference_rating'] = (
        weights['location_score'] * apply_non_linear_transformation(yelp_data['location_score']) +
        weights['business_stars'] * exponential_boost(yelp_data['business_stars_norm']) +
        weights['reviewer_stars'] * angular_score_adjustment(yelp_data['reviewer_stars_norm']) +
        weights['engagement_score'] * np.log1p(yelp_data['engagement_score'])
    )

    # Normalize the preference_rating to 1-10 scale
    rating_scaler = MinMaxScaler(feature_range=(1, 10))
    yelp_data['preference_rating'] = rating_scaler.fit_transform(
        yelp_data[['preference_rating']]
    ).round().astype(int)

    # Convert business_id to float
    yelp_data['business_id'] = yelp_data['business_id'].apply(business_id_to_float)

    return yelp_data

# Data preparation and display
def prepare_final_ranking(yelp_data):
    # Remove duplicates, keeping highest-rated business for each name
    unique_data = yelp_data.sort_values(by='preference_rating', ascending=False).drop_duplicates(subset='name', keep='first')

    # Create ranked dataframe
    ranked_yelp = unique_data[['name', 'preference_rating']].sort_values(by='preference_rating', ascending=False)

    return ranked_yelp

# Process the data
yelp_data = generate_preference_rating(yelp_data)

# Prepare and display ranking
ranked_results = prepare_final_ranking(yelp_data)
print(ranked_results)


Using every feature as input to the training process is not a good idea. Some features are not related to business rating, so let's exclude them from our feature set.
1. Extract the following for the input data `X`: `business_stars`,`review_count`,`checkins`,`reviewer_stars`,`useful`,`funny`, and `cool`.
2. Store the label variable in `Y`.

Let the criteria for a good resturant be a rating greater than **eight**.

In [ ]:
# Initialize X and Y
# ANSWER STARTING HERE

X =
Y =

# ANSWER ENDS HERE

3. Use [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from the Scikit-learn module to split the training data using the proposed 80/20 ratio.

In [ ]:
seed = 42 # always use this seed number as your random_state in the whole notebook
test_size=0.2

# Split your training data and use the seed to get consistent results.
# ANSWER STARTING HERE

X_train, X_test, y_train, y_test =

# ANSWER ENDS HERE

4. **Normalize** both the training and test sets so that the model is trained on features with a consistent scale. **You can use any scaler from the Scikit-learn module to do this.** If we don't normalize the data, the model may struggle to converge during training because some features could dominate the loss function due to their larger scale. By normalizing the data, we ensure that each feature contributes equally to the learning process, improving the model's ability to find an optimal solution. (eg. [StandardScaler documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html))

In [ ]:
# ANSWER STARTING HERE

X_train_scaled =
X_test_scaled =

# ANSWER ENDS HERE


### **Part C (15 Pts Total)**

#### This is going to be the main source of evaluation for the return offer. Be thorough with your model search. You will train a **Logistic Regression (max_iter=1000)** Model, and 3 different classifiers: **KNN**, **DecisionTree**, and **Random Forest**.

##### ***3.1 Load Models (4 pts)***

---

1. Create a python dictionary named `models` that maps the names of the models to their constructors.

In [ ]:
# Initialize the models
# ANSWER STARTING HERE
models = {

         }

# ANSWER ENDS HERE



##### ***3.2 K-fold Cross-Validation (8 pts)***
Using the standardized feature sets, do the following:
1. Given five folds and the object with the specified number of splits, perform **k-folds cross-validation** for **each** model using StratifiedKFold.
2. Print the **name** of the model and **cross validation accuracy** (mean and standard deviation) for each.

[Cross Validation Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

In [ ]:
# Perform k-fold Cross-Validation for each model
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=seed)

for model_name, model in models.items():
    np.random.seed(42)
    # ANSWER STARTING HERE

    # ANSWER ENDS HERE


##### ***3.3 Question: What is the most accurate ML model from above choice? Why? (3 pts)***
ANSWER STARTING HERE

### **Part D (12 Pts Total)**
#### We wants you to deploy of all the models to production. Evaluate all of the models on the heldout test set.

##### ***4.1 Model Evaluation (6 pts)***

---


For each of the models do the following in order to evaluate each of them properly:
- **Train** each of the models
- Print each of the **accuracies** (3 decimal precision) of the models by their **names** in this format: *Accuracy of (model name): (accuracy of the model)*.
- Print a **classification report** with the accuracy, precision, recall and f1-score of each model.

In [ ]:
# Train each model using the training data
for model_name, model in models.items():
    # Set random seed for reproducibility
    np.random.seed(42)
    #ANSWER STARTING HERE


    # ANSWER ENDS HERE

# Evaluate the performance of each model
for model_name, model in models.items():
    np.random.seed(42)

    #ANSWER STARTING HERE
    y_pred =
    accuracy =

    print(f"Accuracy of {model_name}: {accuracy:.3f}") # Your accuracy table header here
    print(classification_report(y_test, y_pred)) # Implement your classification report here
    # ANSWER ENDS HERE


##### ***4.2 Model Interpretation (6 pts)***

---

- **Interpret** the results of the classification models (list the models which got the best scores for Accuracy, Precision, Recall and F1 score).
- **Recommend the best model according to your analysis.** Hopefully your best model has over a 90% accuracy as this will guarantee your return offer.

**Question: Which model according to your analysis was the best at predicting the Resturant Preference Rating? Why? DO NOT just say that the accuracy is better than the rest of the models. That is obvious.**

ANSWER STARTING HERE

### **Part E (5 Pts Total)**

#### Seeing all the effort and hard work, it looks as if most people have done well with this project🎉! With all the input features you have used, there is still no concrete way of figuring out whether a feature is more important than another. We only really used intuition for that. Thus, we shall do some feature engineering to determine this.

##### ***5.1 What are your top 4 features used to predict Our resturant preference rating (5 pts)?***
- Use Random Forest to create a dataframe for **feature importance** named **feature_importance_df**.
- **Extract**, **Sort**, and **Print** the **top 4** features from that dataframe.

Here are some references to help you:

[Feature Importance 1](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.feature_importances_)

[Feature Importance 2](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_)



In [ ]:
# Extract the feature importances from the most accurate model
# ANSWER STARTING HERE
importances =

# Creating a DataFrame to display feature importances
feature_importance_df =

# Displaying feature importances


# ANSWER ENDS HERE


### Part F (14 Points Total)

#Now you'll learn about clustering techniques!

You'll perform data preprocessing, apply dimensionality reduction technique, and use K-means clustering to analyze the data. Finally, you'll visualize the results.



##### ***6.1.1 Elbow Method (2.5 pts)***

In K-Means clustering, we start by randomly initializing k clusters and iteratively adjusting these clusters until they stabilize at an equilibrium point. However, before we can do this, we need to decide how many clusters (k) we should use.

In this part, you will determine the optimal number of clusters for a given dataset using the **Elbow Method**. The goal is to identify the point where adding more clusters results in minimal improvement in clustering quality, known as the "elbow point."

The *distortion* for a given number of clusters $k$ can be represented as:



$$D(k) = \frac{1}{N} \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$

Where:
- $D(k)$ is the distortion for $k$ clusters
- $k$ is the number of clusters
- $C_i$ represents the $i$-th cluster
- $x$ is each data point in the cluster
- $\mu_i$ is the centroid (mean) of the $i$-th cluster
- $\|\cdot\|$ denotes the Euclidean distance


In the Elbow Method, the Inertia is a measure of the sum of squared distances between data points and their respective cluster centers. It's often used to evaluate the compactness of clusters and is defined as:

$$I(k) = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$


Task A: Implement your own distortion function to determine the different numbers of clusters, ranging from 1 to 10. Plot the elbow plot and guess the elbow point $k$. Use X_Train that we calculated in the previous parts for this part.

In [ ]:
distortions = []
inertias = []
K = range(1, 10)

for k in K:
    # ANSWER STARTING HERE



# ANSWER ENDS HERE

To determine the optimal number of clusters, we have to select the value of $k$ at the “elbow”, i.e. the point after which the distortion starts decreasing in a linear fashion. Thus for the given data, we conclude that the optimal number of clusters for the data is 3.


In [ ]:
k_cluster = 3

##### ***6.1.2 KElbowVisualizer (2.5 pts)***
Task B: Use the `KElbowVisualizer` from the `yellowbrick` library to verify the conclusion above. See [Elbow Method
](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html).

In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer

# ANSWER STARTING HERE
# Load the dataset

# Instantiate the clustering model and visualizer


# ANSWER ENDS HERE


##### ***6.1.3 Question: According to the plots above, why do you think we chose K=3? (1 pts)***

ANSWER STARTING HERE

##### ***6.2.1 Extract top features (1 pts)***
Extract the top 4 features you defined in Part E for show its clustering for the following steps. Name your feature dataframe `X`.


In [ ]:
# TODO: Select most relevant features for clustering (exclude 'rating')
data = yelp_data

# ANSWER STARTING HERE
features = [ ]
X =
# ANSWER ENDS HERE

print(X)

We wish to standardize the feature sets in order to make sure that the PCA is not biased by differences in scales within the data. Standardization aids with consistency in the data.

##### ***6.2.2 Normalize the data using StandardScaler. (1 pts)***

- **Standardize** the features and save it as `X_scaled`.

*Hint:* [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
# TODO: Normalize the data using StandardScaler
# ANSWER STARTING HERE
X_scaled =
# ANSWER ENDS HERE


There are varying reasons for using a dimensionality reduction step such as PCA prior to data segmentation. By reducing the number of features, we are improving the performance of our algorithm. In addition, by decreasing the number of features the noise is also reduced.

#### PCA

Principal Component Analysis (PCA) is a dimensionality reduction technique that finds directions (principal components) maximizing variance in a dataset. It's an orthogonal linear transformation mapping data to a new coordinate system [1].

PCA steps for a dataset $X$ with $n$ samples and $p$ features:

1. Center the data.
2. Compute covariance matrix $\Sigma = \frac{1}{n-1}X^TX$.
3. Calculate eigenvectors and eigenvalues of $\Sigma$.
4. Sort eigenvectors by descending eigenvalues.
5. Select top $k$ eigenvectors as new basis.

Project original data onto this new basis for transformed data.

PCA is simple and effective for linear relationships but may not suit complex, non-linear structures [2].

For more in-depth reading on PCA:

[1] Scikit-learn PCA documentation: [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

[2] Jolliffe, I. T. (2002). Principal Component Analysis, Second Edition. Springer Series in Statistics.


##### ***6.3.1 Use a PCA function to return the transformed data. (3 pts)***


In [ ]:
# TODO: Apply PCA and return the transformed data

def apply_pca(X_scaled, n_components=2):
    # ANSWER STARTING HERE


    # ANSWER ENDS HERE
    return pca, X_pca
    # Apply PCA

pca, X_pca = apply_pca(X_scaled)


#### K-Means Clustering

K-Means is a popular unsupervised learning algorithm used for clustering, where the goal is to partition data into K clusters based on feature similarity. The algorithm works by iteratively assigning data points to the nearest centroid (cluster center) and updating centroids to minimize the variance within each cluster. It starts by randomly initializing centroids and repeats the process until the centroids stabilize (convergence). K-Means is efficient for large datasets but sensitive to the choice of K and initial centroids. In your homework, you'll implement K-Means to explore how well it groups your data based on similarity!







##### ***6.4 Apply K-means clustering and return the cluster labels (3 pts)***


In [ ]:
# TODO: Apply K-means clustering and return the cluster labels
def apply_kmeans(X, n_clusters, random_state=seed):
    # ANSWER STARTING HERE


    # ANSWER ENDS HERE
    return labels

# Apply K-means to both PCA results
pca_labels = apply_kmeans(X_pca, n_clusters = k_cluster)



Visualize your clustering!

In [ ]:
## Step 6: Visualization

def plot_clusters(X, labels, rating, title, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(12, 8))
    kmeans = KMeans(n_clusters=k_cluster, random_state=seed).fit(X)
    data['Cluster'] = pd.Categorical(kmeans.labels_)
    sns.scatterplot(x=X[:, 0],y=X[:, 1], hue="Cluster", data=data,s=45,legend="full")
    plt.legend(labels=[features[0], features[1], features[2], features[3]])

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

print(X_pca)
print(X_pca.shape)
print(data['preference_rating'].shape)

# Visualize PCA results
plot_clusters(X_pca, pca_labels, data['preference_rating'],
              'PCA: Clusters and Rating', 'PC1', 'PC2')


# Your analysis here


**You have been promoted!!**

**You are now one of the consulting company's elite employees! 🎊🎉🥂🥳**

![](https://media.giphy.com/media/JIX9t2j0ZTN9S/giphy.gif?cid=790b76111hlwamin05rcv1ujslgjh5o49haj07h4wpuvwkgz&ep=v1_gifs_search&rid=giphy.gif&ct=g)

### Extra Credit ✨ (7 points total)
The matplotlib and seaborn libraries are great tools for visualizing data.

They are used to create legible and complex graphs that are essential in data interpretation and analysis.

##### ***7.1 Visualize the distribution of each feature in the original dataset for different rating classes (3 pts)***
- **Plot** out histograms for each feature. You can use a for loop.
- Label the **title** of each histogram "Distribution of (Feature Name) by Rating"
- Label the **axes** appropriately and display each histogram.
- Display the **y-axis** using a **logarithmic** scale.

In [ ]:
# Dropping text or non-numeric columns
histogram_pd = yelp_data.drop(columns=[
    'business_id', 'name', 'neighborhood', 'latitude', 'longitude','address', 'city', 'state',
    'postal_code', 'categories', 'review_id', 'user_id', 'text'
])

histogram_pd = histogram_pd.iloc[:, list(range(8)) + [-2]]
histogram_pd.head()

# ANSWER STARTING HERE
# Histograms for each feature with rating hue
for feature in histogram_pd.columns[:-1]:  # Exclude 'rating' column
    plt.figure(figsize=(10, 6))

    # Create the histogram

    # Create a custom legend

# ANSWER ENDS HERE


##### ***7.2 Explore correlations between features and Rating ratings (2 pts)***
- Create a **correlation matrix** based on the correlations between features and rating.
- **Display** the matrix.

In [ ]:
# Correlation Matrix
# ANSWER STARTING HERE

# ANSWER ENDS HERE


##### **7.3 *Question*: What have you learned from this correlation map? Why? (2 pts)**
ANSWER RIGHT HERE!
